In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install konlpy

In [3]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [4]:
df = pd.read_csv('/content/drive/MyDrive/새싹2024/뉴스 기사 추천 3/Naver_cleansed_8S.csv',  sep=',')
df = df.dropna()
df

,platform_id,category,publisher,publication_date,title,content,author,article_url,update_date
0,네이버,금융,부산일보,2024.08.01,"[속보] 몬테네그로 항소법원 “권도형, 한국으로 송환해야”",2022년 국내 투자자 20만 명이 3000억 원에 달하는 피해을 입은 테라루나 사...,이정훈,https://n.news.naver.com/mnews/article/082/000...,수정 날짜 없음
1,네이버,금융,뉴스1,2024.08.01,"WB '중진국 함정' 극복 대표 사례로 韓 꼽아…""개도국의 필독서""",ⓒ News1 정윤미 기자 전민 기자 세계은행은 1일 발한 세계개발보고서에서 중진국...,전민 기자,https://n.news.naver.com/mnews/article/421/000...,2024.08.01
2,네이버,금융,파이낸셜뉴스,2024.08.01,"지난달 5대 은행 가계대출, 7조 넘게 늘어",7월말 기준 715조7383억거래회복에 대출막차 수요 겹쳐3년 3개월만에 최대폭 증...,박문수,https://n.news.naver.com/mnews/article/014/000...,수정 날짜 없음
3,네이버,금융,매일경제,2024.08.01,"“여보, 달러 팔고 엔화 사자”...두달만에 1360원대, 엔화는 920원 눈앞",원화 달러 약세엔화 강세 연동해 일시적 13401350원대 가능성추세적 반등으로 보...,임영신,https://n.news.naver.com/mnews/article/009/000...,수정 날짜 없음
4,네이버,금융,한겨레,2024.08.01,"일본은행 ‘깜짝’ 금리 인상, 엔화도 깜짝 상승",1일 원엔 환율 91071원지난 26일 서울 중구 하나은행 위변조대응센터에서 직원이...,남지현,https://n.news.naver.com/mnews/article/028/000...,2024.08.01
...,...,...,...,...,...,...,...,...,...
105634,네이버,경제일반,국민일보,2024.08.31,가명에 영어까지 ‘꼼수’…반도체 업계 황당 인재 경쟁,반도체 업계 인재 영입 경쟁 치열경업금지서약서 써도 무용지물법적 효력 미미 형식적 ...,나경연,https://n.news.naver.com/mnews/article/005/000...,수정 날짜 없음
105635,네이버,경제일반,국민일보,2024.08.31,한남동에도 무신사… ‘서울 패션 5대 성지’ 입성 완료,30일 공식 오픈하는 무신사 스탠다드 한남 플래그십 스토어 매장 1층 내부 모습 무...,이다연,https://n.news.naver.com/mnews/article/005/000...,수정 날짜 없음
105636,네이버,경제일반,국민일보,2024.08.31,"엔비디아 독주 막겠다…네이버, 연내 ‘한국판 쿠다’ 공개",인텔 가우디 기반 SW 플랫폼 구축 예정완성된 SW 하이퍼클로바X에도 활용 계획생태...,나경연,https://n.news.naver.com/mnews/article/005/000...,수정 날짜 없음
105637,네이버,경제일반,중앙SUNDAY,2024.08.31,"""향료·색소 첨가해도 막걸리""…전통 훼손인가 규제 완화인가",양유정 기자 문제는 이렇게 줄어든 세금 혜택이 유사 막걸리 수십 종을 생산하는 몇몇...,오유진,https://n.news.naver.com/mnews/article/353/000...,2024.08.31


In [5]:
df['title']

,title
0,"[속보] 몬테네그로 항소법원 “권도형, 한국으로 송환해야”"
1,"WB '중진국 함정' 극복 대표 사례로 韓 꼽아…""개도국의 필독서"""
2,"지난달 5대 은행 가계대출, 7조 넘게 늘어"
3,"“여보, 달러 팔고 엔화 사자”...두달만에 1360원대, 엔화는 920원 눈앞"
4,"일본은행 ‘깜짝’ 금리 인상, 엔화도 깜짝 상승"
...,...
105634,가명에 영어까지 ‘꼼수’…반도체 업계 황당 인재 경쟁
105635,한남동에도 무신사… ‘서울 패션 5대 성지’ 입성 완료
105636,"엔비디아 독주 막겠다…네이버, 연내 ‘한국판 쿠다’ 공개"
105637,"""향료·색소 첨가해도 막걸리""…전통 훼손인가 규제 완화인가"


In [6]:
df['article_url']

,article_url
0,https://n.news.naver.com/mnews/article/082/000...
1,https://n.news.naver.com/mnews/article/421/000...
2,https://n.news.naver.com/mnews/article/014/000...
3,https://n.news.naver.com/mnews/article/009/000...
4,https://n.news.naver.com/mnews/article/028/000...
...,...
105634,https://n.news.naver.com/mnews/article/005/000...
105635,https://n.news.naver.com/mnews/article/005/000...
105636,https://n.news.naver.com/mnews/article/005/000...
105637,https://n.news.naver.com/mnews/article/353/000...


In [7]:
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [8]:
# cd Mecab-ko-for-Google-Colab

In [9]:
# !bash install_mecab-ko_on_colab_light_220429.sh
# 이부분은 업데이트 될 수 있기에 제작자의 github에서 확인하는 것이 좋다.

In [10]:
mecab = Mecab()

tagged_corpus_list = []

# for index, row in tqdm(df.iterrows(), total=len(df)):
#   text = row['content']
#   tag = row['title']
#   tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

# print('문서의 수 :', len(tagged_corpus_list))
# 데이터 샘플링
sample_df = df.sample(n=10000)  # 예를 들어, 50,000개 샘플로 학습
sample_tagged_corpus_list = [TaggedDocument(tags=[row['title']], words=mecab.morphs(row['content'])) for index, row in sample_df.iterrows()]
print('문서의 수 :', len(sample_tagged_corpus_list))

문서의 수 : 10000


In [11]:
# tagged_corpus_list[0]
sample_tagged_corpus_list[1]

TaggedDocument(words=['최근', '증가', '하', '는', '공사비', '분쟁', '을', '신속', '하', '게', '해결', '하', '기', '위해', '건설', '분쟁', '조정', '위원회', '의', '사무', '를', '국토', '안전', '관리원', '에', '위탁', '하', '는', '방안', '이', '추진', '된다', '권영진', '국민', '의', '힘', '국회의원', '은', '지난', '23', '일', '이', '같', '은', '내용', '의', '건설', '산업', '기본법', '일부', '개정', '법률안', '을', '대발', '의', '했', '다고', '25', '일', '밝혔', '다', '현행법', '은', '정비', '사업', '시행', '으로', '발생', '하', '는', '분쟁', '을', '조정', '하', '기', '위해', '지자체', '에', '도시', '분쟁', '조정', '위원회', '를', '설치', '하', '도록', '하', '고', '있', '다', '그러나', '최근', '공사비', '상승', '으로', '발주', '처', '와', '시공사', '간', '갈등', '이', '급격히', '늘어나', '이', '에', '따른', '조정', '신청', '도', '증가', '하', '고', '있', '는', '실정', '이', '다', '이', '처럼', '공사비', '와', '관련', '된', '갈등', '이', '증가', '하', '면서', '건설', '분쟁', '조정', '위원회', '의', '중요', '성', '이', '커지', '고', '있', '지만', '건설', '분쟁', '조정', '위원회', '사무국', '은', '국토', '부', '내', '담당', '공무원', '이', '다른', '업무', '와', '병행', '하', '고', '있', '어', '분쟁', '해결', '이', '지연', '되', '는', '실정', '이', '다이', '에', '권영진', 

# doc2vec

In [15]:
from gensim.models import doc2vec

In [13]:
# model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# # Vocabulary 빌드
# model.build_vocab(tagged_corpus_list)

# # Doc2Vec 학습
# model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=20)

# 모델 저장
# model.save('dart.doc2vec')

model = doc2vec.Doc2Vec(vector_size=500, alpha=0.025, min_alpha=0.025, workers=8, window=10)
model.build_vocab(sample_tagged_corpus_list)
model.train(sample_tagged_corpus_list, total_examples=model.corpus_count, epochs=20)  # 에폭 수 조정

NameError: name 'model' is not defined

In [ ]:
# 모델 저장
model.save('dart.doc2vec')

# 모델 로드
model = doc2vec.Doc2Vec.load('dart.doc2vec')

In [ ]:
similar_doc = model.dv.most_similar('농심, 수출 전용 라면 공장 부산 녹산공단에 짓는다')
print(similar_doc)

In [ ]:
similar_doc = model.dv.most_similar('美, 자국 빅테크에 과세하려는 캐나다에 분쟁해결 협의 요청')
print(similar_doc)

In [ ]:
# 유사한 문서 찾기
similar_doc = model.dv.most_similar('"광복 영화" 말하니 목록이 좌르륵…한층 똑똑해진 삼성 AI TV')

# 유사한 문서의 정보를 담을 리스트
similar_articles = []

# 유사한 문서의 제목을 기반으로 원래 데이터프레임에서 정보 가져오기
for title, similarity in similar_doc:
    # 제목을 기준으로 데이터프레임에서 해당 문서 정보 가져오기
    article_info = df[df['title'] == title].iloc[0]

    content = article_info['content']
    news_agency = article_info['publisher']
    url = article_info['article_url']

    # 정보 추가
    similar_articles.append({
        'title': title,
        'content': content,
        'news_agency': news_agency,
        'url': url,
        'similarity': similarity
    })

# 유사한 기사 정보 출력
for article in similar_articles:
    print(f"Title: {article['title']}")
    print(f"Content: {article['content']}")
    print(f"publisher: {article['news_agency']}")
    print(f"URL: {article['url']}")
    print(f"Similarity: {article['similarity']}\n")

# train_test_split 모델 평가

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋을 학습 데이터와 테스트 데이터로 분리
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 학습 데이터로 학습
train_tagged_corpus = [TaggedDocument(tags=[row['title']], words=mecab.morphs(row['content'])) for index, row in train_df.iterrows()]
test_tagged_corpus = [TaggedDocument(tags=[row['title']], words=mecab.morphs(row['content'])) for index, row in test_df.iterrows()]

# Doc2Vec 모델 학습
model = doc2vec.Doc2Vec(vector_size=500, alpha=0.025, min_alpha=0.025, workers=8, window=10)
model.build_vocab(train_tagged_corpus)
model.train(train_tagged_corpus, total_examples=model.corpus_count, epochs=20)


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from multiprocessing import Pool

In [ ]:
# # cosine_similarity
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np
# from multiprocessing import Pool

# # 1: 파일 경로로 데이터 로드
# # df = pd.read_csv('/content/drive/MyDrive/새싹2024/뉴스 기사 추천 3/Naver_cleansed_8S.csv', sep=',').head(5000)
# # sample_df = df.sample(n=10000)
# # 2: TF-IDF 벡터라이저를 사용하여 뉴스 제목 및 내용 벡터화
# tfidf = TfidfVectorizer(stop_words='english')
# tfidf_matrix = tfidf.fit_transform(sample_df['content'])

# # 뉴스 ID와 인덱스 매핑
# indices = pd.Series(sample_df.index, index=sample_df['title']).drop_duplicates()
# print(f"뉴스 ID와 인덱스 매핑: {indices}")
# # 유사도 계산 함수
# def calculate_similarity(idx):
#     sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
#     sim_scores = sorted(list(enumerate(sim_scores)), key=lambda x: x[1], reverse=True)
#     top_10_sim_scores = sim_scores[1:11]  # 유사도가 높은 상위 10개의 뉴스만 유지
#     return (idx, top_10_sim_scores)

# # 멀티프로세싱을 사용한 유사도 계산
# with Pool() as pool:
#     similarity_results = pool.map(calculate_similarity, range(len(sample_df)))

# # 유사도 결과를 딕셔너리에 저장
# similarity_dict = {idx: sim_scores for idx, sim_scores in similarity_results}

# # 콘텐츠 기반 뉴스 추천 함수 정의
# def get_content_based_recommendations(title):
#     idx = indices[title]
#     sim_scores = similarity_dict[idx]
#     news_indices = [i[0] for i in sim_scores]
#     return sample_df[['title', 'article_url']].iloc[news_indices]

# # 예시: 특정 뉴스 제목을 기준으로 추천 받기
# print(get_content_based_recommendations("3조원대 새만금 공항·철도 건설사업 세부방안 논의"))

# 1: 파일 경로로 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/새싹2024/뉴스 기사 추천 3/Naver_cleansed_8S.csv', sep=',').head(10000)  # 필요한 경우 주석 제거

# 2: TF-IDF 벡터라이저를 사용하여 뉴스 제목 및 내용 벡터화
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['content'])

# 뉴스 제목과 인덱스 매핑
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

# 유사도 계산 함수
def calculate_similarity(idx):
    sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
    sim_scores = sorted(list(enumerate(sim_scores)), key=lambda x: x[1], reverse=True)
    top_10_sim_scores = sim_scores[1:11]  # 유사도가 높은 상위 10개의 뉴스만 유지 (자기 자신 제외)
    return (idx, top_10_sim_scores)

# 멀티프로세싱을 사용한 유사도 계산
def calculate_all_similarities():
    with Pool() as pool:
        similarity_results = pool.map(calculate_similarity, range(len(df)))
    return {idx: sim_scores for idx, sim_scores in similarity_results}

# 유사도 결과를 딕셔너리에 저장
similarity_dict = calculate_all_similarities()

# 콘텐츠 기반 뉴스 추천 함수 정의
def get_content_based_recommendations(title):
    # 제목이 데이터프레임에 존재하지 않는 경우
    if title not in indices:
        return f"제목 '{title}'이 데이터프레임에 존재하지 않습니다."

    # 제목에 대한 단일 인덱스 추출
    idx = indices[title]

    # `idx`가 Series일 경우 처리 (예: 같은 제목의 뉴스가 여러 개 있을 때)
    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]  # 첫 번째 인덱스 선택

    # 인덱스가 올바른지 확인하고, 유사도 정보가 없는 경우 처리
    if idx not in similarity_dict:
        return f"제목 '{title}'에 대한 유사도 정보가 없습니다."

    # 유사도 기반 추천 뉴스 목록 생성
    sim_scores = similarity_dict[idx]
    news_indices = [i[0] for i in sim_scores]
    recommendations = df[['title', 'content', 'publisher', 'article_url', 'category']].iloc[news_indices]

    # 제목과 실제 뉴스 매칭 확인
    recommendations['matching'] = recommendations['title'].apply(lambda x: x in df['title'].values)

    return recommendations
def evaluate_recommendations(title):
    recommendations = get_content_based_recommendations(title)

    if isinstance(recommendations, str):
        return recommendations  # 제목이 데이터프레임에 존재하지 않는 경우

    # 추천된 기사와 실제 기사 제목 목록
    recommended_titles = recommendations['title'].tolist()

    # 실제 기사 제목 목록 (예: 제목을 기준으로 단순히 추천을 받은 경우)
    actual_title = title

    # 추천 결과와 실제 결과의 비교
    y_true = [1]  # 실제 관련 기사
    y_pred = [1 if actual_title in recommended_titles else 0]  # 추천된 기사 중 실제로 관련성이 있는지 여부

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return {
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }
# 예시: 특정 뉴스 제목을 기준으로 추천 받기
print(get_content_based_recommendations("거래소, 코스닥도 매도 사이드카 발동"))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 시각화를 위한 샘플 데이터 (행렬의 일부만 시각화)
sample_cosine_sim_matrix = cosine_sim_matrix[:50, :50]  # 50x50 행렬로 축소

plt.figure(figsize=(10, 8))
sns.heatmap(sample_cosine_sim_matrix, cmap='viridis', annot=False)
plt.title('Cosine Similarity Matrix Heatmap (Sample)')
plt.xlabel('Article Index')
plt.ylabel('Article Index')
plt.show()


In [ ]:
import numpy as np

# 대각선 값을 NaN으로 설정
mask = np.eye(cosine_sim_matrix.shape[0], dtype=bool)
masked_matrix = np.where(mask, np.nan, cosine_sim_matrix)

# 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(masked_matrix, cmap='viridis', annot=False, square=True, cbar=True, mask=np.isnan(masked_matrix))
plt.title('Cosine Similarity Matrix Heatmap (Excluding Diagonal)')
plt.xlabel('Article Index')
plt.ylabel('Article Index')
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 대각선 값을 NaN으로 설정
mask = np.eye(cosine_sim_matrix.shape[0], dtype=bool)
masked_matrix = np.where(mask, np.nan, cosine_sim_matrix)

# 히트맵의 색상 맵과 값 범위 조정
plt.figure(figsize=(10, 8))
sns.heatmap(masked_matrix, cmap='viridis', annot=False, square=True, cbar=True,
            mask=np.isnan(masked_matrix), vmin=0, vmax=np.nanmax(masked_matrix))
plt.title('Cosine Similarity Matrix Heatmap (Excluding Diagonal)')
plt.xlabel('Article Index')
plt.ylabel('Article Index')
plt.show()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
from multiprocessing import Pool
import pandas as pd

# 1: 파일 경로로 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/새싹2024/뉴스 기사 추천 3/Naver_cleansed_8S.csv', sep=',').head(30000)

# 2: TF-IDF 벡터라이저를 사용하여 뉴스 제목 및 내용 벡터화
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['content'])

# 뉴스 제목과 인덱스 매핑
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

# 유사도 계산 함수
def calculate_similarity(idx):
    sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
    sim_scores = sorted(list(enumerate(sim_scores)), key=lambda x: x[1], reverse=True)
    top_10_sim_scores = sim_scores[1:11]  # 유사도가 높은 상위 10개의 뉴스만 유지 (자기 자신 제외)
    return (idx, top_10_sim_scores)

# 멀티프로세싱을 사용한 유사도 계산
def calculate_all_similarities():
    with Pool() as pool:
        similarity_results = pool.map(calculate_similarity, range(len(df)))
    return {idx: sim_scores for idx, sim_scores in similarity_results}

# 유사도 결과를 딕셔너리에 저장
similarity_dict = calculate_all_similarities()

# 콘텐츠 기반 뉴스 추천 함수 정의
def get_content_based_recommendations(title):
    if title not in indices:
        return f"제목 '{title}'이 데이터프레임에 존재하지 않습니다."

    idx = indices[title]

    # `idx`가 Series일 경우 처리 (예: 같은 제목의 뉴스가 여러 개 있을 때)
    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]

    if idx not in similarity_dict:
        return f"제목 '{title}'에 대한 유사도 정보가 없습니다."

    # 유사도 기반 추천 뉴스 목록 생성
    sim_scores = similarity_dict[idx]
    news_indices = [i[0] for i in sim_scores]
    recommendations = df[['title', 'content', 'publisher', 'article_url', 'category']].iloc[news_indices]

    return recommendations

# 추천 결과를 평가하는 함수 정의
# 평가 함수 정의
def evaluate_recommendations(title):
    recommendations = get_content_based_recommendations(title)

    if isinstance(recommendations, str):
        return recommendations  # 제목이 데이터프레임에 존재하지 않는 경우

    # 제목에 대한 인덱스를 가져와 해당 본문의 벡터화된 데이터를 가져옴
    idx = indices[title]

    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]  # 첫 번째 인덱스 선택

    query_vector = tfidf_matrix[idx]  # 입력된 제목에 해당하는 벡터

    # 추천된 기사의 인덱스와 그들의 벡터화된 본문 데이터를 가져옴
    recommended_indices = recommendations.index.tolist()
    recommended_vectors = tfidf_matrix[recommended_indices]

    # 실제 본문과 추천된 본문 간의 유사도 계산
    sim_scores = cosine_similarity(query_vector, recommended_vectors).flatten()

    # 임계값 설정: 유사도가 일정 수준 이상이면 관련성이 있다고 판단
    threshold = 0.5
    y_true = [1] * len(recommended_indices)  # 추천된 모든 기사들을 관련 기사로 가정
    y_pred = [1 if sim >= threshold else 0 for sim in sim_scores]  # 임계값 이상인 경우 관련성 있는 것으로 판단

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return {
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# 예시: 특정 뉴스 제목을 기준으로 추천받기
test_title = "[속보] 주가 급반등 코스피·코스닥 매수 사이드카 발동…4년 2개월만"
recommendations = get_content_based_recommendations(test_title)
print(recommendations)
print(evaluate_recommendations(test_title))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from multiprocessing import Pool
import pandas as pd
import numpy as np


# TF-IDF 벡터라이저를 사용하여 뉴스 제목 및 내용 벡터화
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['content'])

# 뉴스 제목과 인덱스 매핑
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

# 유사도 계산 함수
def calculate_similarity(idx):
    sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
    sim_scores = sorted(list(enumerate(sim_scores)), key=lambda x: x[1], reverse=True)
    top_10_sim_scores = sim_scores[1:11]  # 유사도가 높은 상위 10개의 뉴스만 유지 (자기 자신 제외)
    return (idx, top_10_sim_scores)

# 멀티프로세싱을 사용한 유사도 계산
def calculate_all_similarities():
    with Pool() as pool:
        similarity_results = pool.map(calculate_similarity, range(len(df)))
    return {idx: sim_scores for idx, sim_scores in similarity_results}

# 유사도 결과를 딕셔너리에 저장
similarity_dict = calculate_all_similarities()

# 콘텐츠 기반 뉴스 추천 함수 정의
def get_content_based_recommendations(title):
    if title not in indices:
        return f"제목 '{title}'이 데이터프레임에 존재하지 않습니다."

    idx = indices[title]

    # `idx`가 Series일 경우 처리 (예: 같은 제목의 뉴스가 여러 개 있을 때)
    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]

    if idx not in similarity_dict:
        return f"제목 '{title}'에 대한 유사도 정보가 없습니다."

    # 유사도 기반 추천 뉴스 목록 생성
    sim_scores = similarity_dict[idx]
    news_indices = [i[0] for i in sim_scores]
    recommendations = df[['title', 'content', 'publisher', 'article_url', 'category']].iloc[news_indices]

    return recommendations
def evaluate_recommendations(title):
    # 1. 추천 결과 가져오기
    recommendations = get_content_based_recommendations(title)

    if isinstance(recommendations, str):
        return recommendations  # 제목이 데이터프레임에 존재하지 않는 경우

    # 2. 입력된 제목에 대한 인덱스를 가져오기
    idx = indices[title]

    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]  # 같은 제목의 뉴스가 여러 개 있을 때 첫 번째 인덱스 선택

    # 3. 제목에 해당하는 벡터를 가져오기
    query_vector = tfidf_matrix[idx]  # 입력된 제목에 해당하는 TF-IDF 벡터

    # 4. 추천된 기사의 인덱스와 벡터화된 본문 데이터를 가져오기
    recommended_indices = recommendations.index.tolist()
    recommended_vectors = tfidf_matrix[recommended_indices]

    # 5. 추천된 기사와 입력된 제목 간의 유사도 계산
    sim_scores = cosine_similarity(query_vector, recommended_vectors).flatten()

    # 6. 군집 분석을 통해 추천의 품질 평가
    all_vectors = tfidf_matrix[np.concatenate(([idx], recommended_indices))]
    kmeans = KMeans(n_clusters=2, random_state=42)  # 두 개의 군집으로 나누기
    kmeans.fit(all_vectors)
    labels = kmeans.labels_
    silhouette_avg = silhouette_score(all_vectors, labels)  # 실루엣 계수 계산

    # 7. Precision@K, Recall@K을 기반으로 평가
    threshold = 0.3
    y_true = [1] * len(recommended_indices)  # 추천된 모든 기사를 관련 기사로 가정
    y_pred = [1 if sim >= threshold else 0 for sim in sim_scores]  # 임계값 이상인 경우 관련성 있는 것으로 판단

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # 평가 결과 반환
    return {
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Silhouette Score': silhouette_avg
    }

# 예시: 특정 뉴스 제목을 기준으로 추천받기
test_title = "[속보] 주가 급반등 코스피·코스닥 매수 사이드카 발동…4년 2개월만"
recommendations = get_content_based_recommendations(test_title)
print(recommendations)
print(evaluate_recommendations(test_title))

In [ ]:
print(df['title'])